# MS Music: Examples and Visualizations

This notebook demonstrates the `ms_music` Python package with extensive visualizations and comparisons of different processing methods and effects.

**Note:** This notebook assumes you have already installed the `ms_music` package and its dependencies.

## 1. Setup and Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Audio, display
import os

# MS Music imports
from ms_music import MSSonifier
from ms_music import utils as ms_utils
from ms_music import additional_sound_modifiers as creative_fx
from ms_music.visualizations import (
    extract_audio_features,
    plot_waveform_comparison,
    plot_spectrogram_comparison,
    plot_frequency_spectrum_comparison,
    plot_difference_spectrogram,
    plot_feature_comparison,
    plot_chromagram_comparison,
    plot_mfcc_evolution,
    plot_3d_spectrogram,
    plot_3d_spectrogram_waterfall,
    plot_mz_to_frequency_mapping,
    plot_scan_progression,
    compute_audio_similarity_matrix,
    plot_similarity_matrices,
    plot_audio_envelope_comparison,
    create_summary_grid
)

# Set style for plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 2. Configuration and Helper Functions

In [2]:
# Configuration
mzml_filename = "mzml_filename"  # Replace with your actual mzML file name
sample_data_dir = "sample_data_dir" # Directory containing sample mzML files
output_audio_dir = "output_audio_dir" # Directory for output audio files
output_viz_dir = "output_visualizations" # Directory for output visualizations

# Create directories if they don't exist
for dir_path in [sample_data_dir, output_audio_dir, output_viz_dir]:
    os.makedirs(dir_path, exist_ok=True)

mzml_filepath = os.path.join(sample_data_dir, mzml_filename)

# Check if the file exists
if not os.path.exists(mzml_filepath):
    print(f"ERROR: mzML file not found at {mzml_filepath}")
    print("Please place your mzML file in the 'sample_data' directory and update 'mzml_filename'.")
else:
    print(f"Using mzML file: {mzml_filepath}")

# Sonification parameters
MS_LEVEL = 1                # MS level to process (1 or 2)
TOTAL_DURATION_MIN = 1      # Shorter for more examples
SAMPLE_RATE = 44100         # Audio sample rate

## 3. Initialize Sonifier and Load Data

In [3]:
sonifier = None
if os.path.exists(mzml_filepath):
    sonifier = MSSonifier(
        filepath=mzml_filepath,
        ms_level=MS_LEVEL,
        total_duration_minutes=TOTAL_DURATION_MIN,
        sample_rate=SAMPLE_RATE
    )
    sonifier.load_and_preprocess_data()
    
    # Display data statistics
    if sonifier.processed_spectra_dfs:
        print(f"\nData Statistics:")
        print(f"  Number of scans: {len(sonifier.processed_spectra_dfs)}")
        print(f"  m/z range: {sonifier.min_mz_overall:.2f} - {sonifier.max_mz_overall:.2f}")
        print(f"  Max intensity: {sonifier.max_intensity_overall:.2e}")
        
        # Count total peaks
        total_peaks = sum(len(df) for df in sonifier.processed_spectra_dfs)
        print(f"  Total peaks across all scans: {total_peaks}")
else:
    print("Skipping sonifier initialization as mzML file is not found.")

## 4. Compare Different Sonification Methods

Let's generate audio using different methods and mappings, then compare them visually.

In [4]:
# Dictionary to store different sonification results
sonification_results = {}

if sonifier and sonifier.processed_spectra_dfs:
    # 1. Standard Gradient Method
    print("\n1. Generating Standard Gradient Audio...")
    sonifier.sonify(method='gradient', method_params={'overlap_percentage': 0.05})
    sonification_results['Gradient (Linear)'] = sonifier.get_current_audio()
    
    # 2. Enhanced Gradient with Inverse Log Mapping
    print("\n2. Generating Enhanced Gradient (Inverse Log)...")
    sonifier.sonify_enhanced('gradient_enhanced', method_params={
        'frequency_mapping': 'inverse_log',
        'freq_range': (200, 4000),
        'overlap_percentage': 0.05
    })
    sonification_results['Gradient (Inverse Log)'] = sonifier.get_current_audio()
    
    # 3. Enhanced Gradient with Power Law Mapping
    print("\n3. Generating Enhanced Gradient (Power Law)...")
    sonifier.sonify_enhanced('gradient_enhanced', method_params={
        'frequency_mapping': 'power_law',
        'freq_range': (200, 4000),
        'overlap_percentage': 0.05
    })
    sonification_results['Gradient (Power Law)'] = sonifier.get_current_audio()
    
    # 4. Musical Quantized - Major Scale
    print("\n4. Generating Musical Quantized (C Major)...")
    sonifier.sonify_quantized(
        base_mapping='inverse_log',
        method_params={
            'scale': 'major',
            'root_note': 'C',
            'freq_range': (261.63, 2093),  # C4 to C7
        }
    )
    sonification_results['Quantized (C Major)'] = sonifier.get_current_audio()
    
    # 5. Musical Quantized - Minor Pentatonic
    print("\n5. Generating Musical Quantized (A Minor Pentatonic)...")
    sonifier.sonify_quantized(
        base_mapping='inverse_log',
        method_params={
            'scale': 'pentatonic_minor',
            'root_note': 'A',
            'freq_range': (55, 1760), # A1 to A6
        }
    )
    sonification_results['Quantized (A Minor Pent)'] = sonifier.get_current_audio()
    
    # 6. ADSR Method
    print("\n6. Generating ADSR Audio...")
    sonifier.sonify(method='adsr', method_params={
        'adsr_settings': {
            'attack_time_pc': 0.02,
            'decay_time_pc': 0.05,
            'sustain_level_pc': 0.7,
            'release_time_pc': 0.1,
            'randomize': False
        }
    })
    sonification_results['ADSR'] = sonifier.get_current_audio()

    # Normalize and display all audio
    for name, audio in sonification_results.items():
        if audio is not None and audio.size > 0:
            max_amp = np.max(np.abs(audio))
            if max_amp > 0:
                sonification_results[name] = audio / max_amp
            print(f"Display: {name}")
            display(Audio(data=audio, rate=SAMPLE_RATE, normalize=True))
    
    print("\nAll sonification methods completed!")
            

## 5. Visualize Method Comparisons

In [5]:
if sonification_results:
    # 1. Waveform Comparison
    fig = plot_waveform_comparison(sonification_results, SAMPLE_RATE, 
                                   "Waveform Comparison - Different Methods")
    plt.savefig(os.path.join(output_viz_dir, "waveform_comparison_methods.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    # 2. Spectrogram Comparison
    # Select subset for clarity
    selected_methods = {
        'Gradient (Linear)': sonification_results['Gradient (Linear)'],
        'Gradient (Inverse Log)': sonification_results['Gradient (Inverse Log)'],
        'Quantized (C Major)': sonification_results['Quantized (C Major)'],
        'ADSR': sonification_results['ADSR']
    }
    fig = plot_spectrogram_comparison(selected_methods, SAMPLE_RATE,
                                     "Spectrogram Comparison - Selected Methods")
    plt.savefig(os.path.join(output_viz_dir, "spectrogram_comparison_methods.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    # 3. Frequency Spectrum Comparison
    fig = plot_frequency_spectrum_comparison(sonification_results, SAMPLE_RATE,
                                           "Frequency Spectrum - All Methods")
    plt.savefig(os.path.join(output_viz_dir, "spectrum_comparison_methods.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    # 4. Feature Analysis
    feature_dict = {}
    for name, audio in sonification_results.items():
        if audio is not None and audio.size > 0:
            feature_dict[name] = extract_audio_features(audio, SAMPLE_RATE)
    
    fig = plot_feature_comparison(feature_dict, "Audio Features - Different Methods")
    plt.savefig(os.path.join(output_viz_dir, "feature_comparison_methods.png"), dpi=300, bbox_inches='tight')
    plt.show()

## 6. Effects Comparison and Visualization

Now let's apply various effects and visualize their impact.

In [6]:
# Use the inverse log gradient as base for effects demonstration
base_audio = sonification_results.get('Gradient (Inverse Log)')
effects_results = {}

if base_audio is not None:
    effects_results['Original'] = base_audio.copy()
    
    # Apply various effects
    effects_to_apply = [
        ('HPSS Harmonic', 'hpss', {'margin': 16, 'harmonic': True, 'percussive': False}),
        ('HPSS Percussive', 'hpss', {'margin': 16, 'harmonic': False, 'percussive': True}),
        ('Butterworth LP 800Hz', 'butterworth_filter', {'cutoff_freq': 800, 'btype': 'low', 'order': 4}),
        ('Butterworth HP 500Hz', 'butterworth_filter', {'cutoff_freq': 500, 'btype': 'high', 'order': 4}),
        ('Notch 1kHz', 'notch_filter', {'notch_freq': 1000, 'quality_factor': 10}),
        ('Chebyshev LP 1.5kHz', 'chebyshev1_filter', {'cutoff_freq': 1500, 'ripple_db': 1, 'btype': 'low', 'order': 4})
    ]
    
    for effect_name, effect_func, params in effects_to_apply:
        print(f"Applying {effect_name}...")
        sonifier.current_audio_data = base_audio.copy()
        sonifier.apply_effect(effect_func, effect_params=params)
        effects_results[effect_name] = sonifier.get_current_audio()
    
    # Visualize effects comparison
    print("\nVisualizing effects comparison...")
    
    # 1. Difference spectrograms for selected effects
    fig = plot_difference_spectrogram(
        effects_results['Original'], 
        effects_results['Butterworth LP 800Hz'],
        SAMPLE_RATE,
        "Original", "Butterworth LP 800Hz"
    )
    plt.savefig(os.path.join(output_viz_dir, "difference_spectrogram_butterworth.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    # 2. Frequency spectrum comparison of filters
    filter_comparison = {
        'Original': effects_results['Original'],
        'LP 800Hz': effects_results['Butterworth LP 800Hz'],
        'HP 500Hz': effects_results['Butterworth HP 500Hz'],
        'Notch 1kHz': effects_results['Notch 1kHz']
    }
    fig = plot_frequency_spectrum_comparison(filter_comparison, SAMPLE_RATE,
                                           "Filter Effects - Frequency Response")
    plt.savefig(os.path.join(output_viz_dir, "filter_frequency_response.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    # 3. Feature comparison for effects
    effects_features = {}
    for name, audio in effects_results.items():
        if audio is not None and audio.size > 0:
            effects_features[name] = extract_audio_features(audio, SAMPLE_RATE)
    
    fig = plot_feature_comparison(effects_features, "Audio Features - Effects Comparison")
    plt.savefig(os.path.join(output_viz_dir, "feature_comparison_effects.png"), dpi=300, bbox_inches='tight')
    plt.show()

## 7. Creative Effects with additional_sound_modifiers

Let's demonstrate the creative effects available in the additional_sound_modifiers module.

In [7]:
if base_audio is not None:
    creative_results = {}
    creative_results['Original'] = base_audio.copy()
    
    # 1. Granular Synthesis
    print("Applying Granular Synthesis...")
    granular_audio = creative_fx.apply_granular_synthesis(
        base_audio, SAMPLE_RATE,
        grain_duration_ms=30,
        density=1.5,
        pitch_variation_semitones=2.0,
        output_duration_factor=1.0
    )
    creative_results['Granular Synthesis'] = granular_audio
    
    # 2. Pitch Shift
    print("Applying Pitch Shift (+3 semitones)...")
    pitch_shifted = creative_fx.apply_pitch_shift(base_audio, SAMPLE_RATE, n_steps=3)
    creative_results['Pitch Shift +3'] = pitch_shifted
    
    # 3. Time Stretch
    print("Applying Time Stretch (0.75x speed)...")
    time_stretched = creative_fx.apply_time_stretch(base_audio, SAMPLE_RATE, rate=0.75)
    creative_results['Time Stretch 0.75x'] = time_stretched[:len(base_audio)]  # Trim to original length
    
    # 4. Reverb
    print("Applying Reverb...")
    reverb_audio = creative_fx.apply_reverb(
        base_audio, SAMPLE_RATE,
        reverb_time_s=0.8,
        dry_wet_mix=0.3
    )
    creative_results['Reverb'] = reverb_audio
    
    # 5. Chorus
    print("Applying Chorus...")
    chorus_audio = creative_fx.apply_chorus(
        base_audio, SAMPLE_RATE,
        delay_ms=25.0,
        depth_ms=3.0,
        rate_hz=0.5,
        dry_wet_mix=0.5,
        num_voices=3
    )
    creative_results['Chorus'] = chorus_audio
    
    # Visualize creative effects
    print("\nVisualizing creative effects...")
    
    # Spectrogram comparison
    creative_subset = {
        'Original': creative_results['Original'],
        'Granular': creative_results['Granular Synthesis'],
        'Reverb': creative_results['Reverb'],
        'Chorus': creative_results['Chorus']
    }
    fig = plot_spectrogram_comparison(creative_subset, SAMPLE_RATE,
                                     "Creative Effects - Spectrogram Comparison")
    plt.savefig(os.path.join(output_viz_dir, "spectrogram_creative_effects.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    # Save some creative effect examples
    for name, audio in creative_results.items():
        if name != 'Original' and audio is not None:
            filename = f"creative_{name.lower().replace(' ', '_')}.wav"
            filepath = os.path.join(output_audio_dir, filename)
            normalized = ms_utils.normalize_audio_to_16bit(audio)
            ms_utils.save_wav(filepath, normalized, SAMPLE_RATE)
            print(f"Saved: {filename}")
            display(Audio(data=audio, rate=SAMPLE_RATE, normalize=False))

## 8. Advanced Visualization: Time-Frequency Analysis

Let's create some advanced visualizations to better understand the sonification process.

In [8]:
# Apply advanced visualizations using imported functions
if sonification_results:
    # 1. Chromagram comparison for musical methods
    musical_methods = {
        'Quantized (C Major)': sonification_results['Quantized (C Major)'],
        'Quantized (A Minor Pent)': sonification_results['Quantized (A Minor Pent)']
    }
    fig = plot_chromagram_comparison(musical_methods, SAMPLE_RATE,
                                    "Chromagram - Musical Scales")
    plt.savefig(os.path.join(output_viz_dir, "chromagram_musical_scales.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    # 2. MFCC Evolution
    if 'Gradient (Inverse Log)' in sonification_results:
        fig = plot_mfcc_evolution(sonification_results['Gradient (Inverse Log)'], 
                                 SAMPLE_RATE,
                                 "MFCC Evolution - Gradient (Inverse Log)")
        plt.savefig(os.path.join(output_viz_dir, "mfcc_evolution.png"), dpi=300, bbox_inches='tight')
        plt.show()
    
    # 3. 3D Spectrogram
    if 'ADSR' in sonification_results:
        fig = plot_3d_spectrogram(sonification_results['ADSR'][int(12*SAMPLE_RATE):int(17*SAMPLE_RATE)],  # First 0.5 seconds
                                 SAMPLE_RATE,
                                 "3D Spectrogram - ADSR Method", max_freq=5000)
        plt.savefig(os.path.join(output_viz_dir, "3d_spectrogram_adsr.png"), dpi=300, bbox_inches='tight')
        plt.show()

    # 4. 3D Spectrogram waterfall
    if 'Quantized (C Major)' in sonification_results:
        fig = plot_3d_spectrogram_waterfall(sonification_results['Quantized (C Major)'][int(12*SAMPLE_RATE):int(17*SAMPLE_RATE)],  # First 0.5 seconds
                                 SAMPLE_RATE,
                                 "3D Spectrogram - Quantized (C Major)", max_freq=5000)
        plt.savefig(os.path.join(output_viz_dir, "3d_spectrogram_quantized_c_major.png"), dpi=300, bbox_inches='tight')
        plt.show()

In [9]:
# Use plot_audio_envelope_comparison and create_summary_grid
fig = plot_audio_envelope_comparison(sonification_results, SAMPLE_RATE,
                                     "Audio Envelope Comparison - All Methods")
plt.savefig(os.path.join(output_viz_dir, "audio_envelope_comparison.png"), dpi=300, bbox_inches='tight')
plt.show()

fig = create_summary_grid(sonifier, sonification_results, SAMPLE_RATE,
                          "Sonification Summary Grid")
plt.savefig(os.path.join(output_viz_dir, "sonification_summary_grid.png"), dpi=300, bbox_inches='tight')
plt.show()

## 9. MS Data to Audio Mapping Visualization

Let's visualize how the MS data maps to audio frequencies.

In [10]:
# Create MS data visualizations using imported functions
if sonifier:
    # 1. m/z to frequency mapping
    fig = plot_mz_to_frequency_mapping(sonifier)
    if fig:
        plt.savefig(os.path.join(output_viz_dir, "mz_frequency_mapping.png"), dpi=300, bbox_inches='tight')
        plt.show()
    
    # 2. Scan progression
    fig = plot_scan_progression(sonifier, num_scans=30)
    if fig:
        plt.savefig(os.path.join(output_viz_dir, "scan_progression.png"), dpi=300, bbox_inches='tight')
        plt.show()

## 10. Correlation Analysis Between Methods

Let's analyze how similar different sonification methods are to each other.

In [11]:
# Compute and plot similarity matrices using imported functions
if sonification_results:
    methods, corr_matrix, spec_matrix = compute_audio_similarity_matrix(sonification_results, SAMPLE_RATE)
    fig = plot_similarity_matrices(methods, corr_matrix, spec_matrix)
    plt.savefig(os.path.join(output_viz_dir, "method_similarity_matrices.png"), dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print insights
    print("\nKey Insights from Similarity Analysis:")
    print("="*50)
    
    # Find most similar pairs
    for i in range(len(methods)):
        for j in range(i+1, len(methods)):
            if spec_matrix[i, j] > 0.9:
                print(f"High spectral similarity: {methods[i]} <-> {methods[j]} ({spec_matrix[i, j]:.3f})")
    
    # Find most different pairs
    print("\nMost different methods (spectral):")
    for i in range(len(methods)):
        for j in range(i+1, len(methods)):
            if spec_matrix[i, j] < 0.5:
                print(f"  {methods[i]} <-> {methods[j]} ({spec_matrix[i, j]:.3f})")

## 11. Export and Summary

Let's create a summary of all generated audio files and visualizations.

In [12]:
# Save all sonification results
print("Saving all sonification results...")
for name, audio in sonification_results.items():
    if audio is not None and audio.size > 0:
        filename = f"method_{name.lower().replace(' ', '_').replace('(', '').replace(')', '')}.wav"
        filepath = os.path.join(output_audio_dir, filename)
        normalized = ms_utils.normalize_audio_to_16bit(audio)
        ms_utils.save_wav(filepath, normalized, SAMPLE_RATE)
        print(f"  Saved: {filename}")

# Create summary report
summary_report = f"""MS Music Sonification Summary Report
=====================================

Input File: {mzml_filename}
MS Level: {MS_LEVEL}
Duration: {TOTAL_DURATION_MIN} minutes
Sample Rate: {SAMPLE_RATE} Hz

Data Statistics:
- Number of scans: {len(sonifier.processed_spectra_dfs) if sonifier else 'N/A'}
- m/z range: {sonifier.min_mz_overall:.2f} - {sonifier.max_mz_overall:.2f} if sonifier else 'N/A'
- Max intensity: {sonifier.max_intensity_overall:.2e} if sonifier else 'N/A'

Sonification Methods Generated:
"""

for method in sonification_results.keys():
    summary_report += f"- {method}\n"

summary_report += f"\n\nTotal audio files generated: {len(sonification_results)}\n"
summary_report += f"Total visualizations created: {len(os.listdir(output_viz_dir))}\n"

# Save summary report
with open(os.path.join(output_audio_dir, "summary_report.txt"), 'w') as f:
    f.write(summary_report)

print("\n" + "="*50)
print(summary_report)
print("="*50)
print(f"\nAll outputs saved to:")
print(f"  Audio: {output_audio_dir}")
print(f"  Visualizations: {output_viz_dir}")